## 文本向量化
1. Gensim库简介
2. 环境安装
3. 词语编码
4. 词向量模型训练
5. 词语相似性计算
6. 词向量模型可视化效果

## Gensim简介
Gensim是一个开源的Python工具包，专注于处理文本数据，将文档表示为语义向量（semantic vectors）。它广泛应用于自然语言处理（NLP）任务，尤其适用于大规模文本语料库的无监督学习。

以下是一些Gensim库的功能和用法：

1. **把文本变成向量**： 词袋模型（Bag-of-Words），TF-IDF（Term Frequency - Inverse Document Frequency）：
2. **训练向量模型**：Word2Vec，Doc2Vec
3. **计算文本相似度**：词与词之间的相似度，文档与文档之间的相似度
4. **挖掘文档的内容主题**：从大量文档中自动识别出隐藏的“主题”


In [ ]:
!pip install gensim

In [ ]:
# Gensim核心概念
#文档（Document）：一些文本。
document = "今天外卖点了爆辣螺狮粉，太好吃了"
#语料库 (Corpus)：文档的集合————用于模型训练的输入
corpus = [
    "今天外卖点了爆辣螺狮粉，太好吃了",
    "昨晚图书馆复习高数到凌晨一点，头都大了",
    "舍友打游戏打到凌晨三点，我快崩溃了",
    "体育课跑了八百米，我差点没缓过来",
    "今天在食堂偶遇暗恋对象，心脏狂跳不止",
    "打工兼职发传单，感觉自己像个社恐战士",
    "被喜欢的人点赞朋友圈，开心到飞起"
]
#向量：将每个文档表示为特征向量。
#用数字来描述文档的“内容”
#向量可以用于计算相似度、分类、聚类等任务
vector_1=[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]
vector_2=[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]

#模型：将向量从一种表示形式转换为另一种表示形式的算法。可将新的词语句子进行编码/提取主题
#gensim.models.TfidfModel 
#gensim.models.Word2Vec
#gensim.models.Doc2Vec
#gensim.models.LdaModel
#gensim.models.FastText
#gensim.models.KeyedVectors
#gensim.models.LsiModel
#gensim.models.Phrases
#gensim.models.Phraser

In [ ]:
#数据预处理
import jieba

corpus = [
    "今天外卖点了爆辣螺狮粉，太好吃了",
    "昨晚图书馆复习高数到凌晨一点，头都大了",
    "舍友打游戏打到凌晨三点，我快崩溃了",
    "体育课跑了八百米，我差点没缓过来",
    "今天在食堂偶遇暗恋对象，心脏狂跳不止",
    "打工兼职发传单，感觉自己像个社恐战士",
    "今天第一次尝试滑板，摔了个屁股墩",
    "室友半夜吃泡面香死我了，忍不住也煮了一包",
    "考前抱佛脚真的有用，我数学竟然及格了",
    "被喜欢的人点赞朋友圈，开心到飞起",
    "实验课做爆试管了，老师脸都黑了"
]

def chinese_preprocess(text, stop_words):
    words = jieba.lcut(text) # 分词
    words = [word for word in words if word not in stop_words] # 去除停用词
    words = [word for word in words if word.isalpha()] # 去除标点符号和数字
    words = [token for token in words if all('\u4e00' <= c <= '\u9fa5' for c in token)] # 去除非中文
    #words = [token for token in words if len(token) > 1] # 根据长度排除短词
    return words

stop_words = set(line.strip() for line in open('resource/中文stoplist.txt', 'r', encoding='utf-8'))
tokenized_corpus = [chinese_preprocess(doc, stop_words) for doc in corpus]

print("清洗后的分词结果：")
tokenized_corpus

In [ ]:
#词袋编码
from gensim import corpora

#创建词典
dictionary = corpora.Dictionary(tokenized_corpus) #每遇到一个新词，就分配一个唯一的整数 ID
#print(dictionary.token2id) #“词到编号”的映射

#将每个文档转换为词袋向量
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
for i, vec in enumerate(bow_corpus):
    print(f"文档{i+1}：{vec}")

In [ ]:
# TF-IDF编码
from gensim.models import TfidfModel
# 训练 TF-IDF 模型
tfidf_model = TfidfModel(bow_corpus) # 使用词袋向量作为输入

# 将原始的词袋向量转化为 TF-IDF 向量
tfidf_corpus = tfidf_model[bow_corpus]

print("\nTF-IDF 向量（每个词的权重）：")
for i, vec in enumerate(tfidf_corpus):
    print(f"文档{i+1}：{vec}")

In [ ]:
# Word2Vec编码
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=tokenized_corpus,  # 输入清洗好的语料
    vector_size=256,     # 每个词向量的维度（推荐：100~300）
    window=5,            # 上下文窗口大小（一个词左右各看几个词）
    min_count=1,         # 忽略出现次数少于 min_count 的词（这里设为1以保留所有词）
    workers=4,           # 并行训练的线程数（根据 CPU 核心数设置）
    sg=1,                # 训练算法：0为CBOW，1为Skip-gram
    epochs=50,            # 训练轮数（迭代整个语料的次数）
    seed=123              # 随机种子，保证每次训练得到的向量相同
)

# 保存模型
model.save("word2vec_model.model")


In [ ]:
# 载入模型
model = Word2Vec.load("word2vec_model.model")
print('词表长度：', len(model.wv))  #输出词向量表个数
print('向量维度：',model.wv.vector_size) #词输出向量维度
print('语料数：', model.corpus_count) #输出语料数

# 找出与某个词最相似的前5个词
analysis_word="今天"
similar_words = model.wv.most_similar(analysis_word, topn=5)
for word, similarity in similar_words:
    print(f"{word}: 相似度={similarity:.4f}")

# 计算两个词之间的余弦相似度
similarity = model.wv.similarity("暗恋", "对象")
print(f"'暗恋' 和 '对象' 的相似度：{similarity:.4f}")

In [ ]:
#加载预训练的word2vec模型
#https://radimrehurek.com/gensim/models/word2vec.html

#下载预训练模型
import gensim
import gensim.downloader as api
##查看所有可能的模型
print(list(gensim.downloader.info()['models'].keys()))
##模型命名<embedding_type>-<data_source>-<extra_info>-<dimensions>
# 例如glove-wiki-gigaword-300
# glove：词向量算法，GloVe（Global Vectors for Word Representation）
# wiki-gigaword：训练语料来源，维基百科 + Gigaword（一个大型新闻语料库）
# 300：词向量维度（embedding dimension）

In [ ]:
import gensim.downloader as api
model = api.load('glove-wiki-gigaword-100') #加载训练好的模型并应用模型
# 验证关键词语是否存在
words_to_check = ['king', 'man', 'woman', 'queen']
for word in words_to_check:
    print(f"'{word}' 存在于模型中: {word in model.key_to_index.keys()}")

# 执行：King - Man + Woman ≈ ?
try:
    result = model.most_similar(
        positive=['king','woman'], # 正例
        negative=['man'], # 负例
        topn=10 
    )
    print("\n结果:")
    for i,(word, similarity) in enumerate(result):
        print(f"Top{i+1}:{word} (相似度: {similarity})")
except KeyError as e:
    print(f"错误: 词语 {e} 不在词汇表中！")

In [ ]:
#首都的示例 paris - france + japan ？
try:
    result = model.most_similar(
        positive=['paris','japan'], # 正例
        negative=['france'], # 负例
        topn=10 
    )
    print("\n结果:")
    for i,(word, similarity) in enumerate(result):
        print(f"Top{i+1}:{word} (相似度: {similarity})")
except KeyError as e:
    print(f"错误: 词语 {e} 不在词汇表中！")

In [ ]:
#dog - bark + meow ？ meow：喵喵叫
try:
    result = model.most_similar(
        positive=['dog','meow'], # 正例 
        negative=['bark'], # 负例
        topn=10 
    )
    print("\n结果:")
    for i,(word, similarity) in enumerate(result):
        print(f"Top{i+1}:{word} (相似度: {similarity})")
except KeyError as e:
    print(f"错误: 词语 {e} 不在词汇表中！")

In [ ]:
#训练自己word2vec模型以红楼梦为例。 林黛玉最相近的词汇有哪些？可视化红楼梦的词分布
# 读取整个文本文件
file_path = "./resource/红楼梦"
import os
#读取路径中所有的文件
chapter_data = []
for file_name in os.listdir(file_path):
    if file_name.endswith(".txt"):
        chapter_path = os.path.join(file_path, file_name)
        with open(chapter_path, 'r', encoding='utf-8') as file:
            text = file.read()
            chapter_data.append((file_name, text))
            print(file_name, text[0:100])

In [ ]:
##数据预处理，对清洗每一章节数据
import jieba
import re
def wash_chinese(text):
    washed_text=[token for token in jieba.cut_for_search(text)] #分词
    stoplist=[line.strip() for line in open(r'resource/中文stoplist.txt').readlines()]
    result=[token for token in washed_text if token not in stoplist]
    result=[token for token in result if not token.isnumeric()] #排除数字
    result=[token for token in result if all(map(lambda c:'\u4e00' <= c <= '\u9fa5',token))]  #排除非中文
    result = [token for token in result if len(token)>1] #排除单字
    return result
##对每一章的内容进行数据预处理
all_clead_data=[]
for title, content in chapter_data:
    cleaned_corpus = wash_chinese(content)
    all_clead_data.append(cleaned_corpus)

print("清洗前：",chapter_data[0][1][:100])
print("清洗后：",all_clead_data[0][:100])

In [ ]:
# 模型训练
from gensim.models import word2vec
train_data=all_clead_data
model = word2vec.Word2Vec(sentences=train_data, sg=1, min_count=5,window=10 ,vector_size=300,epochs=50)
hognlou_model_name="w2v_model_name_hongloumeng"
model.save(hognlou_model_name)

In [ ]:
print(model.wv.vector_size) #词输出向量维度
print('语料数：', model.corpus_count) #输出语料数
print('词表长度：', len(model.wv.key_to_index))  
print('词表内容',model.wv.key_to_index)


In [ ]:
## 查看跟林黛玉最近的100组词
sim_result=model.wv.most_similar(positive=['黛玉'], topn=20)
sim_result

In [ ]:
result = model.wv.most_similar(
    positive=['宝玉','姑娘'], # 正例
    negative=['男子'], # 负例
    topn=10 
)
result

In [ ]:
#可视化词嵌入
#可视化方式1
import csv
from gensim.models import Word2Vec
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#visualising-word-embeddings
  

In [ ]:
!pip install scikit-learn

In [ ]:
#可视化方式1 matplotlib
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

# 这段代码的主要作用：是对训练好的词向量模型进行降维处理，并将结果可视化。
# 定义了一个函数 reduce_dimensions，用于将词向量从高维空间降到二维空间
def reduce_dimensions(model):
    num_dimensions = 2  # 指定最终的维度为2D

    # 提取模型中的词向量和对应的词汇列表
    vectors = np.asarray(model.wv.vectors)  # 获取所有词向量
    labels = np.asarray(model.wv.index_to_key)  # 获取所有词汇

    # 使用 t-SNE 算法对词向量进行降维
    tsne = TSNE(n_components=num_dimensions, random_state=0)  # 初始化 t-SNE 模型
    vectors = tsne.fit_transform(vectors)  # 对词向量进行降维

    # 分离降维后的坐标值
    x_vals = [v[0] for v in vectors]  # 提取降维后每个词向量的第一个维度值
    y_vals = [v[1] for v in vectors]  # 提取降维后每个词向量的第二个维度值
    return x_vals, y_vals, labels  # 返回降维后的坐标值和词汇列表


# 定义了一个函数 plot_with_matplotlib，用于绘制降维后的词向量分布图
def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    # 设置字体为系统支持的中文字体（适用于 macOS）
    plt.rcParams['font.family'] = 'Songti SC'

    # 设置随机种子以保证结果可复现
    random.seed(0)

    # 创建一个 12x12 的绘图区域
    plt.figure(figsize=(12, 12))

    # 绘制散点图
    plt.scatter(x_vals, y_vals)  # 根据降维后的坐标值绘制散点图

    # 随机选择部分词汇进行标注
    indices = list(range(len(labels)))  # 获取所有词汇的索引
    selected_indices = [i for i in range(10)]  # 选择前10个词汇进行标注
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))  # 在散点图上标注词汇

# 加载之前训练好的 Word2Vec 模型
hognlou_model = Word2Vec.load(hognlou_model_name)  

# 对模型中的词向量进行降维处理
x_vals, y_vals, labels = reduce_dimensions(hognlou_model)

# 调用绘制函数，生成词向量分布图
plot_with_matplotlib(x_vals, y_vals, labels)

In [ ]:
## 可视化方式2  https://projector.tensorflow.org/
import csv
def w2v_visial(w2v_model_name,visial_name=""):
    model = Word2Vec.load(w2v_model_name) #加载并应用模型
    vocabs = model.wv.key_to_index.keys()
    print(vocabs)
    for vocab in vocabs:
        word=vocab
        vector=model.wv[vocab]
        with open(f'词向量分析/{visial_name}_w2v_vector.tsv','a',encoding='utf-8',newline='') as f_vec: #提取的特定信息文本
            csv_writer=csv.writer(f_vec,delimiter='\t')
            csv_writer.writerow(vector)
        with open(f'词向量分析/{visial_name}_w2v_word.tsv','a',encoding='utf-8') as f_word: #提取的特定信息文本
            f_word.write(word.strip()+'\n')
#保存词表和对应的向量
w2v_visial(hognlou_model_name,visial_name="honglou")